In [1]:
import numpy as np
import os
import sys
sys.path.append(r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project')

import matplotlib.pyplot as plt # for plotting
%matplotlib widget
import pandas as pd # for data manipulation
from Models.load_data import *
from Models.helpers import *

%load_ext autoreload
%autoreload 2

import torch

path = r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Training_data'
features_list = os.listdir(os.path.join(path, r'features'))

In [2]:
# This cell takes 19s
# Instantiate the dataset without transformations
dataset = IndependentCSVDatasetTCN(path, features_list)

# Split dataset into training and testing sets
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

feature_min, feature_max = compute_global_minmax(train_dataset)
print("Global feature min (train set only):", feature_min)
print("Global feature max (train set only):", feature_max)

global_transform = GlobalMinMaxNormalize(feature_min, feature_max)
train_dataset.dataset.transform = global_transform
test_dataset.dataset.transform = global_transform  # Same transform to prevent data leakage

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)



Skipping JETno87125.csv: sequence length 6001 is unexpected.
Skipping JETno97612.csv: sequence length 6001 is unexpected.
Skipping JETno91071.csv: sequence length 6001 is unexpected.
Skipping JETno99471.csv: sequence length 6001 is unexpected.
Global feature min (train set only): [ 0.0000000e+00 -3.5154782e+06 -6.1130988e+14  0.0000000e+00
 -3.1516222e+14  0.0000000e+00]
Global feature max (train set only): [6.8073063e+00 7.2445923e+03 6.6544650e+17 4.9767345e+16 1.8753850e+18
 1.6388709e+04]


In [3]:
# Create an iterator from the DataLoader
train_iterator = iter(train_loader)

# Get a single batch
batch = next(train_iterator)

# Unpack the batch (assuming your dataset returns (inputs, targets))
inputs, targets = batch

print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)


Inputs shape: torch.Size([32, 6000, 6])
Targets shape: torch.Size([32, 6000])


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


In [4]:
# Define model parameters
input_size = 6  # Number of features per timestep
output_size = 1  # Binary classification per timestep
num_channels = [32, 64, 128]  # Number of channels in each TCN layer
kernel_size = 51
dropout = 0.2


# Initialize the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = TCNModel(input_size, output_size, num_channels, kernel_size, dropout).to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=compute_class_weights(train_loader))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

output = model(inputs.to(device))
print(criterion(output, targets.to(device)))

Class counts: [1394884.0, 21116.0]
Class ratio: 66.05815124511719


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:292: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(class_counts[0] / class_counts[1], dtype=torch.float32)


tensor(0.8900, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [ ]:
import torch
from sklearn.metrics import f1_score

# Number of epochs
num_epochs = 10  

# Move model to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0
    train_correct, train_total = 0, 0
    
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)  # Output shape: (batch_size, sequence_length)

        # Compute loss
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        # Convert predictions to binary values (0 or 1)
        predictions = (output > 0.5).int()
        train_correct += (predictions == y_batch).sum().item()
        train_total += y_batch.numel()  # Total number of timesteps

        # Print loss update for each batch
        print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx+1}/{len(train_loader)}] - Train Loss: {loss.item():.4f}")

    # Compute average training loss and accuracy
    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = train_correct / train_total

    # --- Validation / Test Evaluation ---
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    test_correct, test_total = 0, 0
    all_test_preds, all_test_targets = [], []

    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            test_output = model(X_test)

            # Compute loss
            test_loss += criterion(test_output, y_test).item()
            
            # Convert predictions to binary values (0 or 1)
            test_predictions = (test_output > 0.5).int()
            test_correct += (test_predictions == y_test).sum().item()
            test_total += y_test.numel()
            
            # Store for F1-score computation
            all_test_preds.append(test_predictions.cpu().numpy().flatten())
            all_test_targets.append(y_test.cpu().numpy().flatten())
    
    # Compute average test loss and accuracy
    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = test_correct / test_total
    
    # Compute F1-score
    all_test_preds = np.concatenate(all_test_preds)
    all_test_targets = np.concatenate(all_test_targets)
    test_f1_score = f1_score(all_test_targets, all_test_preds, average='macro')

    # Print progress
    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4%} | "
          f"Test Loss: {avg_test_loss:.4f}, Test Acc: {test_accuracy:.4%}, Test F1: {test_f1_score:.4f}")

print("Training completed.")


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


Epoch [1/10] Batch [1/8] - Train Loss: 0.8982
Epoch [1/10] Batch [2/8] - Train Loss: 0.3562
Epoch [1/10] Batch [3/8] - Train Loss: 8.3231
Epoch [1/10] Batch [4/8] - Train Loss: 0.6846
Epoch [1/10] Batch [5/8] - Train Loss: 0.5938
Epoch [1/10] Batch [6/8] - Train Loss: 1.9368
Epoch [1/10] Batch [7/8] - Train Loss: 0.7174
Epoch [1/10] Batch [8/8] - Train Loss: 4.2357
Epoch [1/10] | Train Loss: 2.2182, Train Acc: 98.5088% | Test Loss: 1.0939, Test Acc: 99.1075%, Test F1: 0.4978


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


Epoch [2/10] Batch [1/8] - Train Loss: 2.0249
Epoch [2/10] Batch [2/8] - Train Loss: 1.5516
Epoch [2/10] Batch [3/8] - Train Loss: 0.6804
